In [2]:
import cvxpy as cp
import numpy as np

In [19]:
T = 5
N = 5

Pod = cp.Variable((N,N))
A = cp.Variable((T, N))
nij = [cp.Variable((N,N)) for _ in range(T)]
fij = [[[cp.Variable((N,N)) for _ in range(T)] for _ in range(N)] for _ in range(N)]

In [20]:
capacity = 100
D = np.array([[0, 2, 2, 0,0],[2,0,2,3,0],[2,2,0,4,1],[0,0,2,0,4],[0,0,2,4,0]])*capacity

F = 5

In [21]:
constraints = []
constraints += [Pod >=0, A>=0]
for t in range(T):
    constraints += [cp.sum(A[t]) == F]
    constraints += [nij[t] >= 0]

    for i in range(N):
        constraints += [nij[t][i,i] == 0]
        if t < T-1:
            constraints += [A[t+1, i] == A[t, i]-cp.sum(nij[t][i,:])+cp.sum(nij[t][:,i])]
        constraints += [A[t,i]>=cp.sum(nij[t][i,:])]
        for j in range(N):
            if i < j:
                constraints += [nij[t][i,j]==-nij[t][j,i]]
                constraints += [fij[n][m][t][i,j]==-fij[n][m][t][j,i] for n in range(N) for m in range(N)]
            constraints += [Pod[i, j]==cp.sum([fij[i][j][t][i,:] for t in range(T)])]
            constraints += [Pod[i, j]==-cp.sum([fij[i][j][t][:,j] for t in range(T)])]
            for k in range(N):
                if k != i and k!= j:
                    constraints += [0==cp.sum([fij[i][j][t][k,:] for t in range(T)])]
            constraints += [cp.sum([fij[n][m][t][i,j] for n in range(N) for m in range(N)])<= nij[t][i,j]*capacity]



In [22]:
obj = cp.Minimize(cp.sum(D-Pod))
prob = cp.Problem(obj, constraints)

prob.solve(verbose=True)

                                     CVXPY                                     
                                     v1.4.2                                    
(CVXPY) Nov 14 09:45:57 AM: Your problem has 3300 variables, 2127 constraints, and 0 parameters.
(CVXPY) Nov 14 09:45:57 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 14 09:45:57 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 14 09:45:57 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Nov 14 09:45:57 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Nov 14 09:45:58 AM: Compiling problem (target solver=MOSEK)

3200.0

In [23]:
ans = np.zeros((N,N))
for t in range(T):
    ans += nij[t].value
    print(nij[t].value)
print(ans)

[[ 5. -0. -0. -0. -0.]
 [-0. -5. -0. -0. -0.]
 [-0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0.]]
[[-5.  0.  0.  0.  0.]
 [-0.  5.  0.  0.  0.]
 [-0. -0. -0.  0.  0.]
 [-0. -0. -0. -0.  0.]
 [-0. -0. -0. -0. -0.]]
[[-0.  0.  0.  0.  0.]
 [-0. -0.  0.  0.  0.]
 [-0. -0. -0.  0.  0.]
 [-0. -0. -0. -0.  0.]
 [-0. -0. -0. -0. -0.]]
[[-0.  0.  0.  0.  0.]
 [-0. -0.  0.  0.  0.]
 [-0. -0. -0.  0.  0.]
 [-0. -0. -0. -0.  0.]
 [-0. -0. -0. -0. -0.]]
[[-0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0.]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [24]:
print(A.value)

[[5. 0. 0. 0. 0.]
 [0. 5. 0. 0. 0.]
 [5. 0. 0. 0. 0.]
 [5. 0. 0. 0. 0.]
 [5. 0. 0. 0. 0.]]


In [25]:
Pod.value

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [13]:
def getOptimal(A):

    p = cp.Variable(2)
    G = cp.Variable()

    constraints = [p>=0, G>=0, cp.sum(p)==1, A.T@p>=G]
    obj = cp.Maximize(G)
    prob = cp.Problem(obj, constraints)
    prob.solve()
    return G.value, p.value

In [14]:
A= np.asarray([[3,1],[0,2]])
getOptimal(A)

(array(1.5), array([0.5, 0.5]))

In [11]:
p.value

array([0.5, 0.5])

In [12]:
A@p.value

array([2., 1.])

In [ ]:
def fillQ(T,Q, u1, u2):
    U = np.zeros(T+1)
    strats = [0]*T
    for t in range(T,0,-1):
        A = np.asarray([[u1(t,0,0), u1(t,0,1)],[u1(t,1,0),u1,(t,1,1)]])
        val, strat = getOptimal(A)
        U[t-1] = val+U[t]
        strats[t-1] = strat

    return U, strats

        
